In [12]:
import numpy as np
import open3d as o3d

### Extrinsic Matrix calculation

In [3]:
def quaternion_rotation_matrix(Q, T):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
    
    tx = T[0]
    ty = T[1]
    tz = T[2]
    
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
    
    translation_matrix = np.array([[tx],
                                   [ty],
                                   [tz]])
                            
    transformation_matrix = getTransformation(rot_matrix, translation_matrix)
    return rot_matrix, translation_matrix, transformation_matrix

def getTransformation(R, t):
    # given Rotation matrix, R and translation matrix, t
    # return the transformation matrix describing the combined motion, T
    augment = np.array([0, 0, 0, 1]) # Converting to homogeneous matrix
    R = np.array(R)
    t = np.array(t)
    if (t.shape == (3,)):
        t = t.reshape((3,1))
    T = np.hstack((R, t))
    T = np.vstack((T, augment))
    assert(T.shape, (4,4))
    return T

# Inverse of a Transformation matrix
def getInverseTransform(T):
    # Given a transformation, T that moves(rotates and translates) a point from A
    # to B, return inv_T transformation that restores the point back to A.
    assert(T.shape, (4,4))
    R = T[:3,:3]
    print('R shape:', R.shape)
    R_inv = R.T
    print('R_inv shape:', R_inv.shape)
    t = T[:3,3]
    t_inv = -1 * np.matmul(R_inv, t)
    t_inv = t_inv.reshape(t_inv.shape[0], -1)
    print('t_inv shape:', t_inv.shape)
    T_inv = np.hstack((R_inv, t_inv))
    augment = np.array([0, 0, 0, 1]) # Converting to homogeneous matrix
    T_inv = np.vstack((T_inv, augment))
    return T_inv

def mapto3D(T, K, K_inv, coordMatrix2D):
    '''
        T: transformation matrix
        K: intrinsic matrix
        K_inv:  inverse of intrinsic matrix
        coordMatrix2D: 2d coordinates = [u v 1]^transpose
    
        coordMatrix3D: 3d coorinates = [x_w y_w z_w 1]^transpose
    '''
    print('T shape:', T.shape)
    coordMatrix3D = None
    T_inv = getInverseTransform(T)
    
    '''
    K_inv = K_inv
    try:
        K_inv = np.linalg.inv(K)
    '''
    
#     print('inv(K) shape:', np.linalg.inv(K).shape)
    print('K_inv shape:', K_inv.shape)
    coordMatrix3D = np.matmul(np.matmul(T_inv, K_inv), coordMatrix2D)
    
    '''
    except np.linalg.LinAlgError:
#         Not invertible. Skip this one.
    print('K is Singular matrix - non invertible')
        pass
    '''
    
    return coordMatrix3D

<>:60: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:67: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:60: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:67: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:60: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:67: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-3-0a23afe0cebe>:60: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(T.shape, (4,4))
<ipython-input-3-0a23afe0cebe>:67: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(T.shape, (4,4))


### Intrinsic Matrix calculation
#### NOTE: Please cross-check below params once

In [4]:
'''
alphaX: focalLength in x in pixels
alphaY: focalLenght in y in pixels
u0 = principal point in X
v0 = principal point in y
gamma: skew coeff between x and y axis. is commonly 0
'''
alphaX = 960
alphaY = 960
u0 = 480
v0 = 360
gamma = 0

import sympy as sp

def determinant(A):
    if len(sp.Matrix(A).rref()[1]) < max(np.shape(A)):
        print('1st condition')
        return 0
    else:
        return np.linalg.det(A)

K = np.array([[alphaX, gamma, u0, 0],
              [0,     alphaY, v0, 0],
              [0,     0,      1,  0]])

K_inv = (1/(alphaX*alphaY))*np.array([[alphaY, gamma, -u0*alphaY, 0],
                                      [0,     alphaX, -v0*alphaX, 0],
                                      [0,     0,      alphaX*alphaY,  0]])
# print(determinant(K))
# print(np.linalg.det(K))
print(np.matmul(K.T,K_inv))

[[ 1.00e+00  0.00e+00 -4.80e+02  0.00e+00]
 [ 0.00e+00  1.00e+00 -3.60e+02  0.00e+00]
 [ 5.00e-01  3.75e-01 -3.74e+02  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00]]


In [12]:
# Bakul1_100 = [0.999624, -0.0232377, -0.00441006, 0.0138727], [0.0877391, -5.68903, -1.18716]
# Bakul1_130 = [0.999578, -0.0241463, -0.0054248, 0.015219], [0.0443816, -5.26152, -1.21401]
# Bakul1_150 = [0.999429, -0.0298655, -0.00258129, 0.015579], [0.0646409, -5.04149, -0.943123]
# Bakul1_390 = [0.999531, -0.0239092, 0.00874164, 0.0170169], [0.116173, -2.21853, -0.475097]
# Bakul1_740 = [0.998457, -0.0203139, 0.0459722, 0.0235951], [-0.00785304, 2.28523, 0.570601]
'''
Input below to quaternion_rotation_matrix() is of a single Image from COLMAP files i.e. Quartenions and Translation matrix  
'''
rotate_matrix, translation_matrix, T = quaternion_rotation_matrix([0.998457, -0.0203139, 0.0459722, 0.0235951], [-0.00785304, 2.28523, 0.570601])

# Bakul1_100 = 2.950000000000000000e+02,3.700000000000000000e+01,4.600000000000000000e+02,1.900000000000000000e+02
# Bakul1_130 = 2.770000000000000000e+02,6.500000000000000000e+01,4.870000000000000000e+02,2.550000000000000000e+02,6.930000000000000000e+02,6.700000000000000000e+01,8.930000000000000000e+02,2.500000000000000000e+02
# Bakul1_150 = 2.860000000000000000e+02,1.100000000000000000e+02,4.910000000000000000e+02,2.920000000000000000e+02,7.030000000000000000e+02,1.070000000000000000e+02,8.960000000000000000e+02,2.870000000000000000e+02
# Bakul1_390 (1st floor window) = [7.370000000000000000e+02,4.370000000000000000e+02,8.920000000000000000e+02,5.500000000000000000e+02],[3.400000000000000000e+02,4.250000000000000000e+02,4.920000000000000000e+02,5.350000000000000000e+02],[3.150000000000000000e+02,0.000000000000000000e+00,5.290000000000000000e+02,1.350000000000000000e+02],[7.480000000000000000e+02,0.000000000000000000e+00,9.510000000000000000e+02,1.320000000000000000e+02]
# Bakul1_740 = [8.020000000000000000e+02,4.700000000000000000e+02,9.570000000000000000e+02,5.820000000000000000e+02], [4.100000000000000000e+02,4.500000000000000000e+02,5.600000000000000000e+02,5.620000000000000000e+02], [3.950000000000000000e+02,1.200000000000000000e+01,5.750000000000000000e+02,1.670000000000000000e+02]
'''
Input below is coordinates of windows in an image from our generated CSV files 
'''
sX, sY, eX, eY = 3.950000000000000000e+02,1.200000000000000000e+01,5.750000000000000000e+02,1.670000000000000000e+02

# start3D = np.matmul(R, np.array([[sX], [sY], [0]])) + t
# end3D = np.matmul(R, np.array([[eX], [eY], [0]])) + t

'''
start2D: start 2D(left top) coordinate of window
end2D: end 2D(right bottom) coordinate of window
start3D: start 3D(left top) coordinate of window
end3D: end 3D(right bottom) coordinate of window
'''
start2D = np.array([[sX], [sY], [1]])
end2D = np.array([[eX], [eY], [1]])

start3D = mapto3D(T, K, K_inv.T, start2D)
end3D = mapto3D(T, K, K_inv.T, end2D)

print('start3D:', start3D)
print('end3D:', end3D)

T shape: (4, 4)
R shape: (3, 3)
R_inv shape: (3, 3)
t_inv shape: (3, 1)
K_inv shape: (4, 3)
T shape: (4, 4)
R shape: (3, 3)
R_inv shape: (3, 3)
t_inv shape: (3, 1)
K_inv shape: (4, 3)
start3D: [[  19.05481648]
 [   7.70985088]
 [-199.94627739]
 [   0.        ]]
end3D: [[  32.98886567]
 [  13.54917042]
 [-347.29875302]
 [   0.        ]]


In [6]:
'''
Bakul130 = Ground Floor left window:
    start3D: [[  -1.34609643]
     [   7.89968922]
     [-161.67688303]
     [   0.        ]]
    end3D: [[  -2.90328446]
     [  16.62776661]
     [-337.70409306]
     [   0.        ]]

Bakul100 = Left window Ground Floor:
    start3D: [[  -1.00235651]
     [   7.50033853]
     [-160.19669187]
     [   0.        ]]
    end3D: [[  -1.96916892]
     [  14.1701656 ]
     [-299.90943928]
     [   0.        ]]

Bakul150 = Left window Ground floor:
    start3D: [[  -0.47362576]
     [  11.05928613]
     [-182.91551128]
     [   0.        ]]
    end3D: [[  -0.97637376]
     [  21.44879993]
     [-353.34851168]
     [   0.        ]]
 
Bakul390 = Ground floor left window
    start3D: [[   6.37436737]
     [  16.02705708]
     [-327.92209898]
     [   0.        ]]
    end3D: [[   8.68079669]
     [  21.70518534]
     [-445.01194611]
     [   0.        ]]
 
Bakul390 = 1st Floor left window:
    start3D: [[   3.19008213]
     [   7.42221003]
     [-156.29164478]
     [   0.        ]]
    end3D: [[   6.30033284]
     [  15.04186151]
     [-313.70182086]
     [   0.        ]]

Bakul_740 = 2nd Floor Left window
    start3D: [[  19.05481648]
     [   7.70985088]
     [-199.94627739]
     [   0.        ]]
    end3D: [[  32.98886567]
     [  13.54917042]
     [-347.29875302]
     [   0.        ]]
'''


###
'''
start3D: [[293.94412671]
 [ 39.49905013]
 [ -0.49987153]]
end3D: [[454.66211901]
 [196.88502064]
 [ -6.27826438]]
'''

'\nstart3D: [[293.94412671]\n [ 39.49905013]\n [ -0.49987153]]\nend3D: [[454.66211901]\n [196.88502064]\n [ -6.27826438]]\n'

### Map Coordinates through file parsing

In [32]:
import csv

path = r'F:\IIIT-H Work\win_det_heatmaps\rrcServerData\SFM\bakul_front_face\COLMAP001\sparse\0\points3D_image4.txt'
with open(path, 'r') as fd:
    reader = csv.reader(fd)
    print('text File content: ', reader)
    rowArr = []
    for i,line in enumerate(reader):
#         print(row)
#         print(row[0].split())
#         print(line)
        if i > 0:
            rowArr.append(list(map(float, line[0].split())))
        else:
            rowArr.append(line[0].split())
#     print(rowArr)

twoDPoints = rowArr[1]
print('No. of 2DPoints: ', len(twoDPoints))

def findNN(list, point2D, searchLimit):
    '''
    Input:
    list- 2D points list = [x, y, 3D_id,....] from COLMAP text file
    point2D- 2Dpoint whose Nearest point is to be found
    searchLimit- Index in global list till which it should be searched    
    '''
    X,Y = point2D
    minDistX = -1
    minDistY = -1
    index = -1
    for i in range(0, searchLimit, 3):
        # remove points not used for 3D reconstruction
        if list[i+2] < 0:
            continue
            
        pt = list[i]
        x,y = list[i], list[i+1]
        distX = (X-x)**2
        distY = (Y-y)**2
        if((minDistX < 0 or minDistY < 0) or (distX < minDistX and distY < minDistY)):
            minDistX = distX 
            minDistY = distY
            index = i
        elif((distX < minDistX and distY > minDistY) or (distX > minDistX and distY < minDistY)):
            # Update if distance is less than min
            minDist = minDistX + minDistY
            dist = distX + distY
            if dist < minDist:
                minDistX = distX 
                minDistY = distY
                index = i
    return index

def findNNv1(list, points_2D, searchLimit):
    '''
    Input:
    list- 2D points list = [x, y, 3D_id,....] from COLMAP text file
    point2D- list of 2Dpoints whose Nearest point are to be found from COLMAP text file
    searchLimit- Index in global list till which it should be searched    
    '''
    indices = []
    for point_2D in points_2D:
        X,Y = point_2D
        minDistX = -1
        minDistY = -1
        index = -1
        for i in range(0, searchLimit, 3):
            # remove points not used for 3D reconstruction
            if list[i+2] < 0:
                continue

            pt = list[i]
            x,y = list[i], list[i+1]
            distX = (X-x)**2
            distY = (Y-y)**2
            if((minDistX < 0 or minDistY < 0) or (distX < minDistX and distY < minDistY)):
                minDistX = distX 
                minDistY = distY
                index = i
            elif((distX < minDistX and distY > minDistY) or (distX > minDistX and distY < minDistY)):
                # Update if distance is less than min
                minDist = minDistX + minDistY
                dist = distX + distY
                if dist < minDist:
                    minDistX = distX 
                    minDistY = distY
                    index = i
        indices.append(index)
    return indices

text File content:  <_csv.reader object at 0x00000148E0A1A5E0>
No. of 2DPoints:  11775


In [33]:
print('List lenght: ', len(twoDPoints))
index = findNN(twoDPoints, (282, 317), len(twoDPoints))
print('index: ', index)
print('x: ' + str(twoDPoints[index]) + ' y: ' + str(twoDPoints[index + 1]) + ' 3D_Index: ' + str(twoDPoints[index + 2]))

#### VERSION 1 ####
indices = findNNv1(twoDPoints, [[694, 137], [889,315], [694,315], [889,137]], len(twoDPoints))
print('indices: ', indices)
for index in indices:
    print('x: ' + str(twoDPoints[index]) + ' y: ' + str(twoDPoints[index + 1]) + ' 3D_Index: ' + str(twoDPoints[index + 2]))

List lenght:  11775
index:  3012
x: 299.189 y: 305.038 3D_Index: 378.0
indices:  [1266, 531, 1284, 2919]
x: 714.345 y: 135.814 3D_Index: 1436.0
x: 877.739 y: 307.668 3D_Index: 132.0
x: 715.981 y: 312.207 3D_Index: 3002.0
x: 891.149 y: 134.365 3D_Index: 1438.0


### Visualizing 3D points

In [23]:
# GROUND FLOOR LEFT WINDOW
'''
x, y, 3D_INDEX
IMage1
309.607 47.0889 4041.0
466.419 192.428 376.0
459.278 47.2349 3932.0
302.345 188.102 131.0

Image2
288.947 58.0746 4137.0
459.243 237.082 376.0
482.018 55.0349 3948.0
290.65 240.111 378.0

Image3
277.54 100.708 976.0
467.516 273.638 376.0
463.58 104.838 1534.0
310.73 280.664 809.0

Image4
291.458 150.562 261.0
463.351 299.734 376.0
476.742 133.093 3231.0
299.189 305.038 378.0
'''

# GROUND FLOOR RIGHT WINDOW
'''
Image2
x: 685.498 y: 59.0552 3D_Index: 4040.0
x: 890.479 y: 261.698 3D_Index: 1110.0
x: 701.649 y: 261.21 3D_Index: 2501.0
x: 889.232 y: 59.0594 3D_Index: 4296.0

Image3
x: 717.741 y: 107.672 3D_Index: 1436.0
x: 876.561 y: 273.765 3D_Index: 1082.0
x: 681.057 y: 278.244 3D_Index: 2685.0
x: 894.45 y: 106.195 3D_Index: 1438.0

Image4
x: 714.345 y: 135.814 3D_Index: 1436.0
x: 877.739 y: 307.668 3D_Index: 132.0
x: 715.981 y: 312.207 3D_Index: 3002.0
x: 891.149 y: 134.365 3D_Index: 1438.0
'''

'\nx, y, 3D_INDEX\nIMage1\n309.607 47.0889 4041.0\n466.419 192.428 376.0\n459.278 47.2349 3932.0\n302.345 188.102 131.0\n\nImage2\n288.947 58.0746 4137.0\n459.243 237.082 376.0\n482.018 55.0349 3948.0\n290.65 240.111 378.0\n\nImage3\n277.54 100.708 976.0\n467.516 273.638 376.0\n463.58 104.838 1534.0\n310.73 280.664 809.0\n\nImage4\n291.458 150.562 261.0\n463.351 299.734 376.0\n476.742 133.093 3231.0\n299.189 305.038 378.0\n'

In [18]:
def displayPointsOnPlane( cloud , Ind ): 
    inlier_cloud = cloud.select_by_index( Ind ) 
    
    window_cloud = cloud.select_by_index( Ind ) # Set to True to save points other than ind
    print("Showing plane points (red) and other (black): ") 
    window_cloud.paint_uniform_color([0, 0, 0])
    cloud.paint_uniform_color([0,1,0])

    for ind in Ind:
        print('Ind: ' + str(ind) + ' point-> ' +  str(np.array(cloud.points)[ind]))

    o3d.visualization.draw_geometries([ window_cloud, cloud ],
                                      width=1080, height=760, zoom=0.1412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])


In [39]:
# 3D indices of ground floor window
list3DIndices_GroundFloorLeft = [131, 376, 378, 3932, 3948, 4137, 4041, 1464, 976, 1534, 809, 261, 3231]
list3DIndices_GroundFloorRight = [4040, 1110, 2501, 4296, 1436, 1438, 1082, 2685, 3002, 132]

pointCloudPath = ".\\bakul_front_face\\COLMAP001\\sparse\\0\\sparse.ply"

# Read the point cloud
cloud = o3d.io.read_point_cloud(pointCloudPath) 

# Downsampling for outlier removal
# cl, ind = cloud.remove_statistical_outlier(nb_neighbors=100, std_ratio=0.1)
# #display_inlier_outlier(cloud, ind)
# cloud_dwn = cloud.select_by_index(ind)
# print("Original Point cloud : Data points : ", cloud_dwn)

# Visualize 3D indices(in black)
# displayPointsOnPlane(cloud, list3DIndices_GroundFloorLeft)
displayPointsOnPlane(cloud, list3DIndices_GroundFloorRight)

Showing plane points (red) and other (black): 
Ind: 4040 point-> [ 0.02842316 -0.12770055  8.99436951]
Ind: 1110 point-> [4.1928978  5.38927889 9.53439617]
Ind: 2501 point-> [-4.44390345  6.55816793 10.16401005]
Ind: 4296 point-> [ 1.30004549 -0.44481334  8.54238605]
Ind: 1436 point-> [4.14686012 5.37841034 9.62307739]
Ind: 1438 point-> [-5.31755543  6.57749796 10.21792221]
Ind: 1082 point-> [-9.13446903  3.8145597  27.23151398]
Ind: 2685 point-> [-4.4504447   2.60084152  9.65375137]
Ind: 3002 point-> [-3.52629876  7.05012465  7.91886044]
Ind: 132 point-> [-1.41573846  3.1174202   9.71073818]
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [20]:
dummyInd = 4041
print(np.array(cloud.points)[dummyInd])

[-2.66384792  3.7582109   9.4654808 ]


## Scaling factor using Camera poses 

In [ ]:
## Bakul 1 new
TY, Height, 1 unit(SFM in cm)
-5.67606 100 
-5.24985 130 70.387 
-5.03129 150 91.508
-4.69864 180 90.184
-4.45071 200 80.667
-4.18163 230 111.49
-3.79717 260 78.0315
-3.52402 280 73.219
-2.86253 340 90.704
-2.51119 370 85.387
-2.21231 390 66.92
-1.86159 420 85.54